# File formats converter
.cooler to .hic

Note that currently .hic doesn't include interchromosomal interatcions.

TODO: include interchromosomal interactions to  output .hic.

In [1]:
import os
import glob

import cooler

java_path   = "java"
juicer_path = "./juicer_tools.1.8.9_jcuda.0.8.jar"

import basic_utils

In [5]:
genome_file = {}
genome_file["hg19"] = "../data/genomes/hg19.reduced.chrom.sizes"

OUT_COOL  = "../data/cool/"
OUT_HIC  = "../data/hic/"
OUT_COMP = "../data/eigenvectors/"
if not os.path.isdir(OUT_HIC):
    os.mkdir(OUT_HIC)
if not os.path.isdir(OUT_COMP):
    os.mkdir(OUT_COMP)
    
files = sorted(glob.glob( OUT_COOL + "*.*" ))

In [7]:
for file in files:
    resolution = int(file.split(".")[-2])
    if resolution!=100000:
        continue
    experiment_id = file.split("/")[-1].split(".")[0]
    cool_file = os.path.join(OUT_COOL, "{}.{}.cool".format(experiment_id, resolution))
    hic_file  = os.path.join(OUT_HIC,  "{}.{}.hic".format(experiment_id, resolution))
    
    c = cooler.Cooler(cool_file)
    genome = genome_file[ c.info["genome-assembly"] ]

    resolutions = [resolution, resolution*2, resolution*5, resolution*10, resolution*100, resolution*1000]
#     basic_utils.cooler2hic(cool_file, hic_file,
#         genome = genome,
#         resolutions = resolutions, 
#         remove_intermediary_files = True,
#         juicer_path = juicer_path,
#         java_path   = java_path)

# Compartments calling
With juicer tools, per chromosome calling.

TODO: implement calling with cooltools, include interchromosomal interactions.

In [10]:
chromnames = cooler.Cooler("../data/cool/A549_NA_NA.100000.cool").chromnames

In [12]:
for file in files:
    resolution = int(file.split(".")[-2])
    if resolution!=100000:
        continue
    experiment_id = file.split("/")[-1].split('.')[0]
    cool_file = os.path.join(OUT_COOL, "{}.{}.cool".format(experiment_id, resolution))
    hic_file  = os.path.join(OUT_HIC,  "{}.{}.hic".format(experiment_id, resolution))
    
    for ch in chromnames:
        
        if ch=="chrM" or ch=="chrY":
            continue
        comp_file  = os.path.join(OUT_COMP,  "{}.{}.{}.comp.bed".format(experiment_id, resolution, ch))
        if os.path.isfile(comp_file):
            continue        
        command = "{} -Xmx2g -jar {} eigenvector -p KR {} {} BP {} {}".format(java_path, juicer_path, hic_file, ch, resolution, comp_file+".txt")
        basic_utils.call_and_check_errors(command)
        
        command = "awk -v OFS='\t' '{{print \"{0}\", {1}*(NR-1), {1}*NR, $1}}' {3} > {2}".format(ch, resolution, comp_file, comp_file+'.txt')
        basic_utils.call_and_check_errors(command)

# Compartments calling of public datasets from AidenLab

For full datasets see: https://aidenlab.org/data.html

In [13]:
data_dict = {"K562": "https://hicfiles.s3.amazonaws.com/hiseq/k562/in-situ/combined.hic",
            "HeLa": "https://hicfiles.s3.amazonaws.com/hiseq/hela/in-situ/combined.hic",
            "HMEC": "https://hicfiles.s3.amazonaws.com/hiseq/hmec/in-situ/combined.hic",
            "NHEK": "https://hicfiles.s3.amazonaws.com/hiseq/nhek/in-situ/combined.hic",
             "HUVEC": "https://hicfiles.s3.amazonaws.com/hiseq/huvec/in-situ/combined.hic",
             "IMR90": "https://hicfiles.s3.amazonaws.com/hiseq/imr90/in-situ/combined.hic",
             "GM12878": "https://hicfiles.s3.amazonaws.com/hiseq/gm12878/in-situ/combined.hic"}

resolution = 100000

for k in data_dict.keys():
    
    experiment_id = k
    hic_file  = data_dict[k]
    
    for ch in chromnames:
        
        if ch=="chrM" or ch=="chrY":
            continue
        comp_file  = os.path.join(OUT_COMP,  "{}.{}.{}.comp.bed".format(experiment_id, resolution, ch))
        
        command = "{} -Xmx2g -jar {} eigenvector -p KR {} {} BP {} {}".format(java_path, juicer_path, hic_file, ch, resolution, comp_file+".txt")
        basic_utils.call_and_check_errors(command)
        
        command = "awk -v OFS='\t' '{{print \"{0}\", {1}*(NR-1), {1}*NR, $1}}' {3} > {2}".format(ch, resolution, comp_file, comp_file+'.txt')
        basic_utils.call_and_check_errors(command)